In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,005 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,347 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,270 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadco

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Read in data from csv
from pyspark.sql import SparkSession
spark = SparkSession.Builder().master("local").appName("thisThing").getOrCreate()
traincleandf = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Proj4/trainclean.csv", header = True)
# Show DataFrame
traincleandf.show()

+------------+---------------+-------------+---------+----------------+-------------------+------------------+---------+---------------+----------------------+----------------+------------------+-------------+
|AnnualIncome|NumBankAccounts|NumCreditCard|NumofLoan|Delayfromduedate|NumofDelayedPayment|NumCreditInquiries|CreditMix|OutstandingDebt|CreditUtilizationRatio|CreditHistoryAge|    MonthlyBalance|       target|
+------------+---------------+-------------+---------+----------------+-------------------+------------------+---------+---------------+----------------------+----------------+------------------+-------------+
|    19114.12|              3|            4|        4|              -1|                0.0|               4.0|     Good|         809.98|     31.94496005538421|             0.0|284.62916249607184|Good/Standard|
|    19114.12|              3|            4|        4|               3|                7.0|               4.0|     Good|         809.98|     28.60935202206993| 

In [67]:
#Schema
traincleandf.printSchema()

root
 |-- AnnualIncome: string (nullable = true)
 |-- NumBankAccounts: string (nullable = true)
 |-- NumCreditCard: string (nullable = true)
 |-- NumofLoan: string (nullable = true)
 |-- Delayfromduedate: string (nullable = true)
 |-- NumofDelayedPayment: string (nullable = true)
 |-- NumCreditInquiries: string (nullable = true)
 |-- CreditMix: string (nullable = true)
 |-- OutstandingDebt: string (nullable = true)
 |-- CreditUtilizationRatio: string (nullable = true)
 |-- CreditHistoryAge: string (nullable = true)
 |-- MonthlyBalance: string (nullable = true)
 |-- CreditScore: string (nullable = true)



In [68]:
#Rename target column as "Credit Score"
traincleandf = df.withColumnRenamed('target', 'CreditScore')
traincleandf.select("CreditScore").show()

+-------------+
|  CreditScore|
+-------------+
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
|Good/Standard|
+-------------+
only showing top 20 rows



In [69]:
#Count frequency of CreditScore values
traincleandf.groupBy('CreditScore').count().show()

+-------------+-----+
|  CreditScore|count|
+-------------+-----+
|Good/Standard|56676|
|         Poor|23129|
+-------------+-----+



In [71]:
#Calculate average Outstanding debt per credit score type
csgroup = traincleandf.groupBy('CreditScore')
csgroup.agg({'OutstandingDebt':'Avg'}).show()

+-------------+--------------------+
|  CreditScore|avg(OutstandingDebt)|
+-------------+--------------------+
|Good/Standard|  1160.8177613098871|
|         Poor|  2082.3704980759976|
+-------------+--------------------+



In [72]:
#Average Credit Utilization Ratio for Credit Score Type
csgroup.agg({'CreditUtilizationRatio':'Avg'}).show()

+-------------+---------------------------+
|  CreditScore|avg(CreditUtilizationRatio)|
+-------------+---------------------------+
|Good/Standard|          32.40054132803124|
|         Poor|          31.99113173075809|
+-------------+---------------------------+



In [ ]:
#Max Credit Utilization Ratio for Credit Score Type
csgroup.agg({'CreditUtilizationRatio':'Max'}).show()

+-------------+---------------------------+
|  CreditScore|max(CreditUtilizationRatio)|
+-------------+---------------------------+
|Good/Standard|          50.00000000000001|
|         Poor|          48.48985172844354|
+-------------+---------------------------+



In [ ]:
# Create a temporary view of the DataFrame.
traincleandf.createOrReplaceTempView('data')

# Look at data using SparkSQL
spark.sql("select * from data limit 10").show()

+------------+---------------+-------------+---------+----------------+-------------------+------------------+---------+---------------+----------------------+----------------+------------------+-------------+
|AnnualIncome|NumBankAccounts|NumCreditCard|NumofLoan|Delayfromduedate|NumofDelayedPayment|NumCreditInquiries|CreditMix|OutstandingDebt|CreditUtilizationRatio|CreditHistoryAge|    MonthlyBalance|  CreditScore|
+------------+---------------+-------------+---------+----------------+-------------------+------------------+---------+---------------+----------------------+----------------+------------------+-------------+
|    19114.12|              3|            4|        4|              -1|                0.0|               4.0|     Good|         809.98|     31.94496005538421|             0.0|284.62916249607184|Good/Standard|
|    19114.12|              3|            4|        4|               3|                7.0|               4.0|     Good|         809.98|     28.60935202206993| 

In [ ]:
#Credit Utilization stats
spark.sql("""
  SELECT CreditScore,
  MAX(CreditUtilizationRatio) as max_CreditUtil,
  Min(CreditUtilizationRatio)as min_CreditUtil,
  Avg(CreditUtilizationRatio) as avg_CreditUtil,
  COUNT(*) AS Frequency
  FROM data
  GROUP BY CreditScore
  """).show()

+-------------+-----------------+------------------+-----------------+---------+
|  CreditScore|   max_CreditUtil|    min_CreditUtil|   avg_CreditUtil|Frequency|
+-------------+-----------------+------------------+-----------------+---------+
|Good/Standard|50.00000000000001|              20.0|32.40054132803124|    56676|
|         Poor|48.48985172844354|20.172941895510625|31.99113173075809|    23129|
+-------------+-----------------+------------------+-----------------+---------+



In [51]:
#Look at records with highest Credit Utilization ratios
traincleandf.filter(df["CreditUtilizationRatio"] > 47).select("CreditUtilizationRatio","CreditScore").show()

+----------------------+-------------+
|CreditUtilizationRatio|  CreditScore|
+----------------------+-------------+
|     50.00000000000001|Good/Standard|
|     49.52232429787243|Good/Standard|
|    48.176598902462246|Good/Standard|
|     48.48985172844354|         Poor|
|     49.56451934738699|Good/Standard|
|     48.19982397892475|Good/Standard|
|     49.06427745162148|Good/Standard|
|    48.337290911787214|Good/Standard|
+----------------------+-------------+



In [55]:
# Average Delay from due date values per Credit Score type
spark.sql("""
  SELECT CreditScore,
  Avg(Delayfromduedate),
  COUNT(*) AS Frequency
  FROM data
  GROUP BY CreditScore
  """).show()

+-------------+---------------------+---------+
|  CreditScore|avg(Delayfromduedate)|Frequency|
+-------------+---------------------+---------+
|Good/Standard|   17.505628484720162|    56676|
|         Poor|     29.7825673397034|    23129|
+-------------+---------------------+---------+



In [66]:
# Average number of loans per credit score type
spark.sql("""
  SELECT CreditScore,
  Avg(NumofLoan),
  COUNT(*) AS Frequency
  FROM data
  GROUP BY CreditScore
  """).show()

+-------------+------------------+---------+
|  CreditScore|    avg(NumofLoan)|Frequency|
+-------------+------------------+---------+
|Good/Standard|2.6140165149269534|    56676|
|         Poor|3.8509663193393577|    23129|
+-------------+------------------+---------+



In [59]:
# Average Income per Credit Score Type
spark.sql("""
  SELECT CreditScore,
  avg(AnnualIncome),
  COUNT(*) AS Frequency
  FROM data
  GROUP BY CreditScore
  """).show()

+-------------+------------------+---------+
|  CreditScore| avg(AnnualIncome)|Frequency|
+-------------+------------------+---------+
|Good/Standard|187032.22164805036|    56676|
|         Poor|153348.41911777537|    23129|
+-------------+------------------+---------+



In [61]:
# Average Credit History age per Credit Score type
spark.sql("""
  SELECT CreditScore,
  avg(CreditHistoryAge),
  COUNT(*) AS Frequency
  FROM data
  GROUP BY CreditScore
  """).show()

+-------------+---------------------+---------+
|  CreditScore|avg(CreditHistoryAge)|Frequency|
+-------------+---------------------+---------+
|Good/Standard|   17.929335168325217|    56676|
|         Poor|   12.457434389727183|    23129|
+-------------+---------------------+---------+

